# EID 444 Homework 2

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

#[1] https://stackoverflow.com/questions/29241056/how-do-i-use-np-newaxis

#R is a matrix of particle positions for N L-J particles



def displacements(R):
    '''Calculates all displacements between all pairs 
    of particles'''

    #Computes are pairwise displacements: r_i - r_j
    #[1] - Learned about np.newaxis to help nduct matrix operations withour consuming much memory
    dR = R[:, np.newaxis, :] - R[np.newaxis, :, :]

    return dR



R = np.array([[1, 2], [3, 4]])  # Two particles

dR = displacements(R)

print(dR)







[[[ 0  0]
  [-2 -2]]

 [[ 2  2]
  [ 0  0]]]


# LJ Forces Function

In [ ]:
def lj_forces(dR, sigma, epsilon):

     rij = np.linalg.norm(dR, axis=-1)
     r_unit_vec = dR / rij

     term_1 = -1 * r_unit_vec * (4 * epsilon / (sigma))
     term_2 = -12 * ((sigma / rij)**13)
     term_3 = 6 * ((sigma / rij)**7)

     FRR = term_1 * (term_2 + term_3)

     return FRR


# Net Forces


In [ ]:
def net_forces(FRR):

    FR = np.sum(FRR, axis=1)
    return FR

# Verlet step

In [ ]:
def verlet_step(r_current, r_prev, t, m , FR):
    ''' Calculate the verlet step of the pendulum-spring 
    system'''

    #Use precomputed values instead of calling function to save computation time
    FRR = lj_forces(dR, sigma, epsilon)
    FR = net_forces(FRR)
    acceleration = FR / m 

    r_next = (2*r_current) - r_prev + ((acceleration) * (t**2))

    return r_next

# Simulation

In [ ]:
def simulation(R_0, v_0, m, t, t_max, sigma, epsilon):

    n_steps = int(t_max / t)
    trajectory = np.empty((n_steps,2))

    r_prev = R_0
    r_current = r_prev + (v_0 * t)

    trajectory[0] = r_prev
    trajectory[1] = r_current

    for step in range(2, n_steps):
        
        r_next = verlet_step(trajectory[step - 1], trajectory[step - 2], t, m, FR)
        trajectory[step] = r_next
        r_previous = trajectory[step - 1]
    
    return trajectory

# Plot and Animate Trajectory

In [ ]:
def plot_traj(trajectory):
    ''' Plots the trajectory on a grid'''

    plt.figure()
    plt.plot(trajectory[:, 0], trajectory[:, 1], label = 'Trajectory')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Simulated Trajectory')
    plt.axis('equal')
    plt.legend()
    plt.show()


def animate_traj(trajectory):
    '''Animates trajectory and saves it as a GIF'''

    fig,ax = plt.subplots()
    ax.set_xlim(np.min(trajectory[:, 0]) - 1, np.max(trajectory[:, 0]) + 1)
    ax.set_ylim(np.min(trajectory[:, 1]) - 1, np.max(trajectory[:, 1]) + 1)
    ax.set_aspect('equal')

    point, = ax.plot([], [], 'bo')  # The mass
    spring, = ax.plot([], [], 'r-', lw=2)  # The spring

    def update(frame):
        point.set_data(trajectory[frame, 0], trajectory[frame, 1])
        spring.set_data([0, trajectory[frame, 0]], [0, trajectory[frame, 1]])
        return point, spring

    ani = animation.FuncAnimation(fig, update, frames=range(0, len(trajectory), 10), blit=True, interval=50)
    ani.save("trajectory.gif", writer="pillow")
    plt.show()









